<a href="https://colab.research.google.com/github/akvo/usaid-wssh-tool-3/blob/main/scripts/data-processing-collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Prologue
%%capture
# @markdown 1. Import Dependencies
import geopandas as gpd
import requests
from io import StringIO
import os
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from IPython.display import display, clear_output, IFrame
from folium import Choropleth
from IPython.display import IFrame
import requests as req

# @markdown 2. Clone Repository
if not os.path.exists("usaid-wssh-tool-3"):
  !git clone https://github.com/akvo/usaid-wssh-tool-3.git

geojson_path = "usaid-wssh-tool-3/data/output"
file_list = [f for f in os.listdir(geojson_path) if os.path.isfile(os.path.join(geojson_path, f))]
file_list = list(filter(lambda x: x.endswith(".geojson"), file_list))

# @markdown 3. Column Definitions
column_definitions = {
    'weighted_drr_raw': 'Weighted_drought_risk',
    'weighted_rfr_raw': 'Weighted_riverine_flood_risk',
    'weighted_bws_raw' : 'Weighted_base_water_stress',
    'od_percent_mean': 'Open_defecation_population_mean',
    'od_population_sum': 'Open_defecation_population_sum',
    'no_improved_water_premise_population_sum': 'No_improved_water_on_premises_population_sum',
    'no_improved_water_premise_population_mean': 'No_improved_water_on_premises_population_mean',
    'no_basic_water_sum': 'No_basic_water_population_sum',
    'no_basic_water_mean': 'No_basic_water_population_mean',
    'no_basic_sanitation_sum': 'No_basic_sanitation_population_sum',
    'no_basic_sanitation_mean': 'No_basic_sanitation_population_mean',
}
column_list = list(column_definitions.keys())

In [2]:
# @title Select Countries
country_list = list(map(lambda x: x.replace(".geojson", ""), file_list))
country_list.sort()

# Display the multi-select widget with full width
multi_select = widgets.SelectMultiple(
    options=[c.capitalize() for c in country_list],
    description='Countries',
    disabled=False,
    layout=widgets.Layout(width='400px', height='200px')  # Set custom width and height
)
display(multi_select)

# Function to print selected items
def on_selection_change(change):
    selected_values = list(multi_select.value)
    print("You selected:", selected_values)

# Attach an observer to capture changes in selection
multi_select.observe(on_selection_change, names="value")

SelectMultiple(description='Countries', layout=Layout(height='200px', width='400px'), options=('Uganda',), val…

In [24]:
#@title Select Indicators
df = gpd.read_file("https://raw.githubusercontent.com/akvo/usaid-wssh-tool-3/refs/heads/main/data/output/uganda.geojson")
scaler = MinMaxScaler()
df[column_list] = scaler.fit_transform(df[column_list])
df.rename(columns=column_definitions, inplace=True)
indicator_selection = widgets.SelectMultiple(
    options=column_list,
    description='Indicators',
    disabled=False,
    layout=widgets.Layout(width='400px', height='200px'),
    style={'description_width': 'initial'}
)

display(indicator_selection)

indicator_checkboxes = {}
for indicator in column_list:
    indicator_checkboxes[indicator] = widgets.Checkbox(
        value=True,
        description=column_definitions[indicator],
        disabled=False,
        indent=False
    )
    display(indicator_checkboxes[indicator])

core_secondary_widgets = {}
directionality_widgets = {}

output_container = []

def on_indicator_change(change):
    clear_output(wait=True)
    output_container.clear()
    core_secondary_widgets.clear()
    directionality_widgets.clear()
    selected_indicators_set = list(indicator_selection.value)
    for indicator in selected_indicators_set:
        indicator_name = column_definitions[indicator]
        if indicator not in core_secondary_widgets:
            core_secondary_widgets[indicator] = widgets.Dropdown(
                options=['core', 'secondary'],
                description=f'{indicator_name} Core/Secondary',
                value='core',  # default value
                layout=widgets.Layout(width='1200px', alignment='right'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        if indicator not in directionality_widgets:
            directionality_widgets[indicator] = widgets.Dropdown(
                options=['positive', 'negative'],
                description=f'{indicator_name} Direction',
                value='positive',  # default value
                layout=widgets.Layout(width='1200px', alignment='right'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        # Add both widgets (Core/Secondary and Directionality) to a VBox
        output_container.append(widgets.VBox([core_secondary_widgets[indicator], directionality_widgets[indicator]]))

    return display(indicator_selection, widgets.VBox(output_container))


indicator_selection.observe(on_indicator_change, names="value")

SelectMultiple(description='Indicators', layout=Layout(height='200px', width='400px'), options=('weighted_drr_…

Checkbox(value=True, description='Weighted_drought_risk', indent=False)

Checkbox(value=True, description='Weighted_riverine_flood_risk', indent=False)

Checkbox(value=True, description='Weighted_base_water_stress', indent=False)

Checkbox(value=True, description='Open_defecation_population_mean', indent=False)

...

Checkbox(value=True, description='No_improved_water_on_premises_population_mean', indent=False)

Checkbox(value=True, description='No_basic_water_population_sum', indent=False)

Checkbox(value=True, description='No_basic_water_population_mean', indent=False)

Checkbox(value=True, description='No_basic_sanitation_population_sum', indent=False)

Checkbox(value=True, description='No_basic_sanitation_population_mean', indent=False)

## Normalize

## Create an interactive tool for calculating the index

In [65]:
# Define the indicator options
indicator_options = geojson_uganda.columns.tolist()

# Create widgets for selecting indicators
indicator_selection = widgets.SelectMultiple(
    options=indicator_options,
    description='Indicators',
    disabled=False,
    layout=widgets.Layout(width='400px', height='200px')  # Set custom width and height

)

# Dynamic widgets for core/secondary and directionality selection
core_secondary_widgets = {}
directionality_widgets = {}

# Store selected indicators to accumulate selections
selected_indicators_set = set()

# Main output area to hold all widgets
output_area = widgets.Output()

def create_widgets_for_indicators(indicators):
    core_secondary_widgets.clear()
    directionality_widgets.clear()

    # Output container to hold all widgets
    output_container = []

    for indicator in indicators:
        if indicator not in core_secondary_widgets:
            core_secondary_widgets[indicator] = widgets.Dropdown(
                options=['core', 'secondary'],
                description=f'{indicator} Core/Secondary',
                value='core',  # default value
                layout=widgets.Layout(width='500px'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        if indicator not in directionality_widgets:
            directionality_widgets[indicator] = widgets.Dropdown(
                options=['positive', 'negative'],
                description=f'{indicator} Direction',
                value='positive',  # default value
                layout=widgets.Layout(width='500px'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        # Add both widgets (Core/Secondary and Directionality) to a VBox
        output_container.append(widgets.VBox([core_secondary_widgets[indicator], directionality_widgets[indicator]]))

    return widgets.VBox(output_container)


# Weights: Core = 0.7, Secondary = 0.3
weight_mapping = {'core': 0.7, 'secondary': 0.3}

In [66]:
# Create a button to update indicator widgets dynamically
update_button = widgets.Button(description="Update Indicators")

#Create a delete button
delete_button = widgets.Button(description="Delete Selected Indicator")

# Create a button to calculate the index
calculate_button = widgets.Button(description="Calculate Index")

In [67]:
# Create output areas to control what gets cleared
output_area = widgets.Output()  # For displaying buttons and widgets
map_area = widgets.Output()     # For displaying the map

In [68]:
def apply_directionality(data, indicators, directionality):
    directed_data = {}
    for idx, indicator in enumerate(indicators):
        if directionality[indicator] == 'positive':
            directed_data[indicator] = data[indicator]  # Positive direction, keep the value as is
        else:
            directed_data[indicator] = 1 - data[indicator]  # Negative direction, invert the value (1 - value)
    return directed_data

In [69]:
# Function to calculate the weighted index and show on the map
def calculate_index(indicators, core_secondary, directionality):
    directed_data = apply_directionality(geojson_uganda, indicators, directionality)
    geojson_uganda['index'] = 0

    for indicator in indicators:
        weight = weight_mapping[core_secondary[indicator]]
        geojson_uganda['index'] += directed_data[indicator] * weight

    # Clear the map area to prevent stacking maps
    with map_area:
        clear_output(wait=True)

    # Create a choropleth map with folium
    m = folium.Map(location=[1.3733, 32.2903], zoom_start=7)

    Choropleth(
        geo_data=geojson_uganda.to_json(),
        data=geojson_uganda,
        columns=["NAME_2", "index"],
        key_on="feature.properties.NAME_2",
        fill_color="YlGnBu",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="WASH Index"
    ).add_to(m)

    # Add a tooltip to display the index score on hover
    folium.GeoJson(
        geojson_uganda,
        style_function=lambda x: {'fillColor': '#ffffff00', 'color': '#00000000', 'weight': 0},
        tooltip=folium.GeoJsonTooltip(fields=["NAME_2", "index"],
                                      aliases=["Region: ", "Index Score: "],
                                      localize=True)
    ).add_to(m)

    # Add a custom legend with "Low priority" and "High priority" labels
    legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 90px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     padding: 10px;">
     <b>Priority Legend</b><br>
     <i style="background: #ffffcc; width: 20px; height: 10px; display: inline-block;"></i> Low priority<br>
     <i style="background: #08306b; width: 20px; height: 10px; display: inline-block;"></i> High priority<br>
     </div>
     '''
    m.get_root().html.add_child(folium.Element(legend_html))

    display(m)
    # Save the map to an HTML file and display it in an IFrame
    #map_file = 'index_map.html'
    #m.save(map_file)
    #display(IFrame(map_file, width=700, height=500))

In [70]:
# Function to handle the button click event for updating widgets
def on_update_button_click(b):
    # Append selected indicators to the set to avoid removing previous selections
    selected_indicators_set.update(list(indicator_selection.value))

    # Clear the output area (not map) and create widgets for each selected indicator
    with output_area:
        clear_output(wait=True)
        # Display indicator list, update button, and delete button
        display(indicator_selection, update_button, delete_button, calculate_button)
        # Display the Core/Secondary and Directionality dropdowns
        display(create_widgets_for_indicators(selected_indicators_set))
        # Display the calculate index button
        #display(calculate_button)


    # Function to handle the button click event for deleting widgets
def on_delete_button_click(b):
    # Remove selected indicators from the set
    selected_indicators_set.difference_update(list(indicator_selection.value))

    # Clear the output area (not map) and update widgets after deletion
    with output_area:
        clear_output(wait=True)
        # Display indicator list, update button, and delete button
        display(indicator_selection, update_button, delete_button)
        # Display the Core/Secondary and Directionality dropdowns
        display(create_widgets_for_indicators(selected_indicators_set))
        # Display the calculate index button
        display(calculate_button)

# Function to display widgets for the selected indicators
def display_widgets(indicators):
    clear_output(wait=True)  # Clear previous output
    create_widgets_for_indicators(indicators)

    display(indicator_selection, update_button, delete_button)  # Re-display the selection and update button

    for indicator in indicators:
        display(core_secondary_widgets[indicator])
        display(directionality_widgets[indicator])

    display(calculate_button)  # Show the calculate button after displaying all the widgets

# Function to handle the button click event for calculating the index
def on_calculate_button_click(b):
    selected_indicators = list(indicator_selection.value)

    # Collect core/secondary and directionality choices
    core_sec = {indicator: core_secondary_widgets[indicator].value for indicator in selected_indicators}
    directionality = {indicator: directionality_widgets[indicator].value for indicator in selected_indicators}

    calculate_index(selected_indicators, core_sec, directionality)

In [71]:
# Attach the button click events to the handler functions
update_button.on_click(on_update_button_click)
delete_button.on_click(on_delete_button_click)
calculate_button.on_click(on_calculate_button_click)

In [72]:
# Display the initial selection widget and buttons
with output_area:
    display(indicator_selection, update_button, delete_button, calculate_button)

# Display both the control widgets and the map area
display(output_area, map_area)

Output()

Output()